In [1]:
import pandas as pd
import numpy as np
import time

In [2]:
timeMatrix = pd.read_csv('C:/Users/Aidan/OneDrive - Simon Fraser University (1sfu)/Garbage Route Optimization/timeMatrixInflated.csv', index_col=0)

In [38]:
zones = pd.read_csv('C:/Users/Aidan/OneDrive - Simon Fraser University (1sfu)/Garbage Route Optimization/poco-allzones.csv', index_col=0)
zones.index = zones.index + ', Port Coquitlam, BC, Canada'

In [39]:
zones = zones[(zones['Garbage Zone'] == 2)]

In [40]:
row = pd.Series({'Garbage Zone':2.0},name='1737 Broadway St Port Coquitlam, BC, Canada')

In [41]:
zones = zones.append(row)

In [42]:
timeMatrixFiltered = timeMatrix.iloc[timeMatrix.index.isin(zones.index),timeMatrix.index.isin(zones.index)]

In [43]:
np.where(timeMatrixFiltered.index == '1737 Broadway St Port Coquitlam, BC, Canada')[0][0]

2093

In [44]:
elements = list(range(0,len(timeMatrixFiltered)))
# elements = list(range(0,16))
# elements.append(2093)

In [45]:
timeMatrixFiltered = timeMatrixFiltered.iloc[elements,elements]

In [46]:
np.where(timeMatrixFiltered.index == '1737 Broadway St Port Coquitlam, BC, Canada')[0][0]

2093

In [47]:
timeMatrix.index[41]

'1735 LINCOLN AVE, Port Coquitlam, BC, Canada'

In [51]:
timeWindows = []
for i in range(len(timeMatrixFiltered)):
    house = timeMatrixFiltered.index[i].split(' ')
    street = ' '.join(house[1:-4])
    
 
    if street == 'ST THOMAS ST,' or street == 'LINCOLN AVE,':
            timeWindows.append((0, 5400))
    else:
        timeWindows.append((0, 50000))

# timeWindows = []
# for i in range(17):
#     house = timeMatrixFiltered.index[i].split(' ')
#     street = ' '.join(house[1:-4])
    
 
#     if street == 'LINCOLN AVE,':
#         timeWindows.append((0,30))
#     else:
#         timeWindows.append((0,50000))

In [55]:
np.where(timeMatrixFiltered.index == '1737 Broadway St Port Coquitlam, BC, Canada')

(array([2093], dtype=int64),)

In [57]:
"""Vehicles Routing Problem (VRP) with Time Windows."""

from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

# def create_data_model(timeMatrix, numVehicles, depotLocation, timeWindows):
#     """Stores the data for the problem."""
#     data = {'time_matrix': timeMatrix.to_numpy(), 
#             'num_vehicles': numVehicles, 
#             'depot': depotLocation, 
#             'time_windows':timeWindows}
#     return data

def solver_status(status):
    if status == 0:
        return 'ROUTING_NOT_SOLVED: Problem not solved yet'
    elif status == 1:
        return 'ROUTING_SUCCESS: Problem solved successfully'
    elif status == 2:
        return 'ROUTING_FAIL: No solution found to the problem'
    elif status == 3:
        return 'ROUTING_FAIL_TIMEOUT: Time limit reached before finding a solution'
    elif status == 4:
        return 'ROUTING_INVALID: Model, model parameters, or flags are not valid'
    else:
        return "Unknown Status"


def create_data_model(timeMatrix, numVehicles, depot, timeWindows):
    """Stores the data for the problem."""
    data = {}
    data['time_matrix'] = timeMatrix.to_numpy()
    data['time_windows'] = timeWindows
    data['num_vehicles'] = numVehicles
    data['depot'] = depot
    return data


def print_solution(data, manager, routing, solution):
    routes = {}
    """Prints solution on console."""
    time_dimension = routing.GetDimensionOrDie('Time')
    total_time = 0
    for vehicle_id in range(data['num_vehicles']):
        routes[vehicle_id] = []
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        while not routing.IsEnd(index):
            routes[vehicle_id].append(manager.IndexToNode(index))
            time_var = time_dimension.CumulVar(index)
            plan_output += '{0} Time({1},{2}) -> '.format(
                manager.IndexToNode(index), solution.Min(time_var),
                solution.Max(time_var))
            index = solution.Value(routing.NextVar(index))
        routes[vehicle_id].append(manager.IndexToNode(index))
        time_var = time_dimension.CumulVar(index)
        plan_output += '{0} Time({1},{2})\n'.format(manager.IndexToNode(index),
                                                    solution.Min(time_var),
                                                    solution.Max(time_var))
        plan_output += 'Time of the route: {}min\n'.format(
            solution.Min(time_var))
        print(plan_output)
        total_time += solution.Min(time_var)
    print('Total time of all routes: {}min'.format(total_time))
    return routes


def main():
    """Solve the VRP with time windows."""
    # Instantiate the data problem.
    data = create_data_model(timeMatrixFiltered,3, 2093, timeWindows)
    
    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data['time_matrix']),
                                           data['num_vehicles'], data['depot'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)


    # Create and register a transit callback.
    def time_callback(from_index, to_index):
        """Returns the travel time between the two nodes."""
        # Convert from routing variable Index to time matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['time_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(time_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Add Time Windows constraint.
    time = 'Time'
    routing.AddDimension(
        transit_callback_index,
        30,  # allow waiting time
        30000,  # maximum time per vehicle
        False,  # Don't force start cumul to zero.
        time)
    time_dimension = routing.GetDimensionOrDie(time)
    # Add time window constraints for each location except depot.
    for location_idx, time_window in enumerate(data['time_windows']):
        if location_idx == data['depot']:
            continue
        index = manager.NodeToIndex(location_idx)
        time_dimension.CumulVar(index).SetRange(time_window[0], time_window[1])
    # Add time window constraints for each vehicle start node.
    depot_idx = data['depot']
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        time_dimension.CumulVar(index).SetRange(
            data['time_windows'][depot_idx][0],
            data['time_windows'][depot_idx][1])

    # Instantiate route start and end times to produce feasible times.
    for i in range(data['num_vehicles']):
        routing.AddVariableMinimizedByFinalizer(
            time_dimension.CumulVar(routing.Start(i)))
        routing.AddVariableMinimizedByFinalizer(
            time_dimension.CumulVar(routing.End(i)))

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
    
    search_parameters.time_limit.seconds = 300

    search_parameters.solution_limit = 1

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)
    
    print("Solver status:", solver_status(routing.status()))

    # Print solution on console.
    if solution:
        return print_solution(data, manager, routing, solution)


if __name__ == '__main__':
    routes = main()

Solver status: ROUTING_SUCCESS: Problem solved successfully
Route for vehicle 0:
2093 Time(0,0) -> 2093 Time(0,0)
Time of the route: 0min

Route for vehicle 1:
2093 Time(0,0) -> 2227 Time(492,492) -> 2112 Time(498,498) -> 238 Time(505,505) -> 306 Time(508,508) -> 183 Time(510,510) -> 1301 Time(512,512) -> 1500 Time(516,516) -> 372 Time(518,518) -> 1008 Time(519,519) -> 2164 Time(521,521) -> 494 Time(523,523) -> 11 Time(538,538) -> 401 Time(540,540) -> 1833 Time(542,542) -> 2161 Time(544,544) -> 1028 Time(545,545) -> 541 Time(547,547) -> 1538 Time(549,549) -> 1232 Time(551,551) -> 381 Time(553,553) -> 438 Time(555,555) -> 227 Time(559,559) -> 237 Time(561,561) -> 115 Time(563,563) -> 1971 Time(565,565) -> 1776 Time(691,691) -> 1810 Time(695,695) -> 642 Time(701,701) -> 428 Time(704,704) -> 215 Time(706,706) -> 1123 Time(707,707) -> 2202 Time(711,711) -> 1443 Time(712,712) -> 1969 Time(714,714) -> 40 Time(720,720) -> 2093 Time(1218,1218)
Time of the route: 1218min

Route for vehicle 2:
2

In [ ]:
def route_names(routes, locations):
    namedRoutes = {}
    for vehicle in routes.keys():
        namedRoutes[vehicle] = locations[routes[vehicle]]
    return namedRoutes
    
namedRoutes = route_names(routes, timeMatrixFiltered.index)

In [ ]:
namedRoutes[1]

In [ ]:
locations = pd.read_csv('C:/Users/Aidan/OneDrive - Simon Fraser University (1sfu)/Garbage Route Optimization/locations.csv', index_col=0)

In [ ]:
tableauRoutes = pd.DataFrame(columns={'address','latitude','longitude','pairID','vehicleID'})
for vehicle in namedRoutes.keys():
    for vertex in range(len(namedRoutes[vehicle])-1):
        currentVertexInfo = locations[locations.index == namedRoutes[vehicle][vertex]]
        currentVertex = {'address':currentVertexInfo.index[0],
                       'latitude':currentVertexInfo['lat'].values[0],
                       'longitude':currentVertexInfo['long'].values[0],
                       'pairID':vertex,
                       'vehicleID':vehicle}

        nextVertexInfo = locations[locations.index == namedRoutes[vehicle][vertex+1]]

        nextVertex = {'address':nextVertexInfo.index[0],
                       'latitude':nextVertexInfo['lat'].values[0],
                       'longitude':nextVertexInfo['long'].values[0],
                       'pairID':vertex,
                       'vehicleID':vehicle}

        tableauRoutes = tableauRoutes.append([currentVertex, nextVertex],ignore_index=True)
        
    
        
        

        
        
    

In [ ]:
tableauRoutes.to_csv('zone1.csv')